In [27]:
%pip install -q langchain-gigachat

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires urllib3<2.0.0,>=1.25.8, but you have urllib3 2.3.0 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import ElasticSearchBM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_gigachat.chat_models import GigaChat
from langchain_core.output_parsers.string import StrOutputParser

In [48]:
embeddings = HuggingFaceEmbeddings(
    model_name="ai-forever/sbert_large_nlu_ru",
    model_kwargs={"device": "cpu"},
    encode_kwargs={'normalize_embeddings': False},
)

In [49]:
client = Elasticsearch(
    hosts="http://localhost:9200",
    basic_auth=("elastic", "password")
)

vector_store = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="tyuiu_index",
    embedding=embeddings,
    es_user="elastic",
    es_password="password",
)

vector_store_retriever = vector_store.as_retriever(search_kwargs={"k": 2})

bm25_retriever = ElasticSearchBM25Retriever(
    client=client, 
    index_name="docs-tyuiu-index",
    search_kwargs={"k": 2}
)

In [50]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_store_retriever, bm25_retriever],
    weights=[0.6, 0.4]
)

In [51]:
docs = ensemble_retriever.invoke("Расскажи о ТИУ")
for doc in docs:
    print(doc)

page_content='Минимальный балл О-ЗФО:'
page_content='Минимальный балл ЗФО:'
page_content='Основные сведения:
Полное наименование образовательной организации
Федеральное государственное бюджетное образовательное учреждение высшего образования «Тюменский индустриальный университет»
Наименование организации на иностранном языке
Federal State Budget Educational Institution of Higher Education «Industrial University of Tyumen»/ IUT
Сокращенное наименование образовательной организации (при наличии)
ФГБОУ ВО «Тюменский индустриальный университет», ФГБОУ ВО «ТИУ», Тюменский индустриальный университет, ТИУ
Дата создания образовательной организации: 08 января 1964 г.
Краткая информация об образовательной организации:
Тюменский индустриальный университет – опорный вуз региона, лидер в подготовке инженерных кадров.
Информация о месте нахождения образовательной организации:
Юридический адрес: 625000, Уральский федеральный округ, Тюменская область, г. Тюмень, ул. Володарского, 38
Фактический адрес: 

In [52]:
prompt = ChatPromptTemplate.from_template(
    template=open("Сотрудник приёмной комиссии.txt", encoding="utf-8").read()
)

In [53]:
model = GigaChat(
    credentials="MDhkMDQ5NDMtODM1Zi00MjNkLTljOGYtZTJmMjMwYjcwNDY2OmY5Y2QwZjNiLWQ4ZDAtNDNjYS1iNjE5LTA1ZTgzZTFiMDU1NQ==",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Max",
    verify_ssl_certs=False,
    profanity_check=False
)

In [54]:
parser = StrOutputParser()

In [55]:
from langchain_core.documents import Document


def parse_docs_content(docs: list[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in docs])

In [56]:
from langchain_core.runnables import RunnablePassthrough


chain = (
    {
        "context": ensemble_retriever | parse_docs_content,
        "question": RunnablePassthrough()
    } |
    prompt |
    model |
    parser
)

In [80]:
answer = chain.invoke("На какие направления нужно сдавть дополнительные испытания")
answer

'Дополнительные вступительные испытания требуются для следующих направлений:\n\n1. **"Архитектура"** (07.03.01):\n   - Композиция (минимум 40 баллов)\n   - Рисунок (минимум 40 баллов)\n   \n2. **"Дизайн архитектурной среды"** (07.03.03):\n   - Композиция (минимум 40 баллов)\n   - Рисунок (минимум 40 баллов)\n\n3. **"Дизайн"** (54.03.01):\n   - Цветографическая композиция (минимум 40 баллов)\n\nЭти испытания проводятся в форме письменной работы на русском языке.'